In [32]:
from uavsar_pytools import UavsarCollection
import pandas as pd
import numpy as np

In [33]:
c = UavsarCollection(collection = 'Grand Mesa, CO', work_dir = '~/')

In [34]:
c.find_urls()

INFO:uavsar_pytools.uavsar_collection:Found 47 image pairs


In [37]:
us_dates = pd.DataFrame()
for r in c.results:
    print(r.properties)
    s = pd.to_datetime(r.properties['startTime'])
    e = pd.to_datetime(r.properties['stopTime'])
    if s > pd.to_datetime('2019-12-01').tz_localize('UTC'):
        us_dates = us_dates.append({'flight1':s, 'flight2':e}, ignore_index= True)

{'beamModeType': 'RPI', 'browse': ['https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090HH_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090HH_01.hgt.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090HV_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090HV_01.hgt.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090VH_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090VH_01.hgt.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090VV_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/grmesa_27416_21019-017_21021-005_0006d_s01_L090VV_01.hgt.png'], 'bytes': '1112873332', 'centerLat': '39.0545480565216', 'centerLon': '-108.110831843488', 'faradayRotation': None, '

/tmp/ipykernel_740/452698778.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  us_dates = us_dates.append({'flight1':s, 'flight2':e}, ignore_index= True)
/tmp/ipykernel_740/452698778.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  us_dates = us_dates.append({'flight1':s, 'flight2':e}, ignore_index= True)
/tmp/ipykernel_740/452698778.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  us_dates = us_dates.append({'flight1':s, 'flight2':e}, ignore_index= True)
/tmp/ipykernel_740/452698778.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  us_dates = us_dates.append({'flight1':s, 'flight2':e}, ignore_index= True)
/tmp/ipykernel_740/4

In [39]:
us_dates = us_dates.sort_values(by = 'flight1').reset_index(drop=True)

In [31]:
us_dates.to_csv('gm_uavsar_flightdates.csv')

Map of snowpit locations

In [45]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# Import our class for the points table
from snowexsql.data import PointData

# Import a useful function to format that data into a dataframe
from snowexsql.conversions import query_to_geopandas

# Import some tools to build dates 
from datetime import date

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)

In [77]:
# Pick a dataset
dataset = 'depth'

# Pick a date
collection_date = date(2020, 2, 7)

# The part inside the query function is what we want back, in this case all columns for the point data
qry = session.query(PointData)

# We then want to filter by the selected the data type depth.
qry = qry.filter(PointData.type == 'depth')

# # Filter by a date
# qry = qry.filter(PointData.date == collection_date)

# Filter to pits
qry = qry.filter(PointData.instrument == 'pit ruler')

# Limit it to a couple thousand 
qry = qry.limit(10000)

# Execute the query and convert to geopandas in one handy function
df = query_to_geopandas(qry, engine)

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

167 records returned!


In [81]:
pd.value_counts(df.date)

2020-02-04    18
2020-01-31    17
2020-01-30    16
2020-02-05    15
2020-02-01    14
2020-01-29    13
2020-01-28    13
2020-02-08    10
2020-02-06     9
2020-02-10     9
2020-02-11     9
2020-02-12     7
2020-03-12     4
2020-02-09     4
2020-01-22     3
2020-03-18     2
2020-03-21     2
2020-02-03     1
2019-12-19     1
Name: date, dtype: int64

In [50]:
pd.to_datetime(df.iloc[0]['date']).date()

datetime.date(2020, 3, 12)

In [74]:
res = pd.DataFrame()
for us_d in us_dates.flight1:
    for _, r in df.iterrows():
        pit_d = pd.to_datetime(r.date).date()
        if pit_d == us_d.date() or pit_d == us_d.date() + pd.to_timedelta('1 day') or pit_d == us_d.date() - pd.to_timedelta('1 day'):
            res = res.append(r)

/tmp/ipykernel_740/3571933736.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(r)
/tmp/ipykernel_740/3571933736.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(r)
/tmp/ipykernel_740/3571933736.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(r)
/tmp/ipykernel_740/3571933736.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(r)
/tmp/ipykernel_740/3571933736.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(r)
/tmp/ipykernel_740/3571933736.py:6: FutureWar

In [85]:
pd.value_counts(res.date)

2020-01-30    64
2020-02-01    56
2020-02-10     9
2020-02-12     7
2020-02-03     4
Name: date, dtype: int64

In [83]:
# Get the Matplotlib Axes object from the dataframe object, color the points by snow depth value
ax = res.plot(column='date', legend=True, cmap='PuBu')

# Use non-scientific notation for x and y ticks
# ax.ticklabel_format(style='plain', useOffset=False)

# # Set the various plots x/y labels and title.
# ax.set_title(f'{len(df.index)} {dataset.title()}s collected on {collection_date.strftime("%Y-%m-%d")}')
# ax.set_xlabel('Easting [m]')
# ax.set_ylabel('Northing [m]')

# # Close the session to avoid hanging transactions
# session.close()

ValueError: Date ordinal 4544200.65 converts to 14411-08-07T15:36:00.000064 (using epoch 1970-01-01T00:00:00), but Matplotlib dates must be between year 0001 and 9999.